In [1]:

#from scipy import sparse

import autotime
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib
time: 650 ms


In [3]:
import regex
import json
import bz2
from tqdm import tqdm

time: 15.3 ms


In [4]:
responses = []
with bz2.BZ2File('D:\\Python\\banki_resp_text\\banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:50, 1817.64it/s]

time: 1min 50s


#  Домашнее задание по NLP # 1 [100 баллов]
## Классификация по тональности 

В этом домашнем задании вам предстоит классифицировать по тональности отзывы на банки с сайта banki.ru.

Данные содержат непосредственно тексты отзывов, некоторую дополнительную информацию, а также оценку по шкале от 1 до 5. 

Тексты хранятся в json-ах в массиве responses.

Посмотрим на пример отзыва:

In [5]:
responses[99]

{'city': 'г. Саратов',
 'rating_not_checked': False,
 'title': 'Карта ко вкладу',
 'num_comments': 0,
 'bank_license': 'лицензия № 880',
 'author': 'ronnichka',
 'bank_name': 'Югра',
 'datetime': '2015-06-03 20:56:57',
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'rating_grade': 3}

time: 2.85 ms


In [6]:
data = pd.DataFrame(responses)
data.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


time: 520 ms


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153499 entries, 0 to 153498
Data columns (total 10 columns):
city                  138325 non-null object
rating_not_checked    153499 non-null bool
title                 153499 non-null object
num_comments          153499 non-null int64
bank_license          153498 non-null object
author                153479 non-null object
bank_name             153499 non-null object
datetime              153499 non-null object
text                  153499 non-null object
rating_grade          88658 non-null float64
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 10.7+ MB
time: 114 ms


In [8]:
data['city'].fillna('неизвестно', inplace = True)
data.drop_duplicates(keep = 'first', inplace = True)

time: 3.37 s


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153499 entries, 0 to 153498
Data columns (total 10 columns):
city                  153499 non-null object
rating_not_checked    153499 non-null bool
title                 153499 non-null object
num_comments          153499 non-null int64
bank_license          153498 non-null object
author                153479 non-null object
bank_name             153499 non-null object
datetime              153499 non-null object
text                  153499 non-null object
rating_grade          88658 non-null float64
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 11.9+ MB
time: 93.1 ms


In [10]:
data.to_csv('D:\\Python\\banki_resp_text\\banki_responses_edit.csv', index=False, index_label=False)
del data, responses

time: 9.19 s


## Часть 1. Анализ текстов [40/100]

1. Посчитайте количество отзывов в разных городах и на разные банки
2. Постройте гистограмы длин слов в символах и в словах
3. Найдите 10 самых частых:
    * слов
    * слов без стоп-слов
    * лемм 
* существительных
4. Постройте кривые Ципфа и Хипса
5. Ответьте на следующие вопросы:
    * какое слово встречается чаще, "сотрудник" или "клиент"?
    * сколько раз встречается слова "мошенничество" и "доверие"?
6. В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру $tf-idf$, для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)

### 1. Посчитайте количество отзывов в разных городах и на разные банки

In [11]:
dataset = pd.read_csv('D:\\Python\\banki_resp_text\\banki_responses_edit.csv')
dataset.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


time: 6.01 s


In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153499 entries, 0 to 153498
Data columns (total 10 columns):
city                  153499 non-null object
rating_not_checked    153499 non-null bool
title                 152276 non-null object
num_comments          153499 non-null int64
bank_license          153498 non-null object
author                153474 non-null object
bank_name             153499 non-null object
datetime              153499 non-null object
text                  153499 non-null object
rating_grade          88658 non-null float64
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 10.7+ MB
time: 61.9 ms


In [13]:
dataset.describe()

,num_comments,rating_grade
count,153499.000000,88658.000000
mean,3.722102,2.153274
std,6.695089,1.509953
min,0.000000,1.000000
25%,0.000000,1.000000
50%,2.000000,1.000000
75%,4.000000,3.000000
max,406.000000,5.000000


time: 24.9 ms


In [14]:
#заполняю пропуски в dataset['title'] - нужно, что бы выделить города из заголовков далее
dataset['title'].fillna('неизвестно', inplace = True)

time: 12.2 ms


In [15]:
city = dataset.city.unique()
bank = dataset.bank_name.unique()
print('Unique citys: ', len(city),'\n', 'Unique banks: ', len(bank))

Unique citys:  5824 
 Unique banks:  670
time: 34.8 ms


In [16]:
#смотрю на список городов более пристально
for city in city:
    print(city)

г. Москва
г. Новосибирск
г. Ставрополь
г. Челябинск
г. Волгоград
г. Пермь
г. Ростов-на-Дону
г. Воронеж
г. Таганрог (Ростовская обл.)
г. Санкт-Петербург
г. Конаково
г. Мурманск
г. Волгодонск (Ростовская обл.)
г. Большеречье
г. Одинцово (Московская обл.)
г. Салават (Республика Башкортостан)
г. Краснодар
г. Кострома
г. Нижний Новгород
г. Орел
г. Саратов
г. Казань
г. Реутов (Московская обл.)
г. Ульяновск
г. Красноярск
г. Хабаровск
г. Магнитогорск (Челябинская обл.)
г. Фролово (Волгоградская обл.)
г. Новочеркасск
г. Чита
г. таганрог
г. Йошкар-Ола
г. Нижний Тагил
г. Ханты-Мансийск
г. Россия
г. Липецк
г. Нижнекамск (Республика Татарстан)
г. Тюмень
г. Москва, Санкт-Петербург
г. Кизел
г. Мончегорск
г. Екатеринбург
г. Омск
г. Астрахань
г. Псков
г. Сочи (Краснодарский край)
г. Комсомольск на амуре
г. Нижний Тагил (Свердловская обл.)
г. Чапаевск
г. УрФО
г. Балаково
г. Курганинск
г. Новокузнецк (Кемеровская обл.)
г. Владивосток
г. Владимир
г. Вологда
г. Ярославль
г. Барнаул
г. Таганрог
г. Южно-Саха

г. Новошахтинск (Ростовская обл.)
г. Ржев
г. Петебург
г. Кузнецк
г. Благовещенск (Республика Башкортостан)
г. Громово
г. Одинцово (городской округ Домодедово)
г. Кимры
г. иркутск
г. Саянск (Иркутская обл.)
г. Чудово
г. Льгов (Курская обл.)
г. Зеленогорск (Красноярский край)
г. Губкин (Белгородская обл.)
г. Усть-Лабинск
г. Лабинск
г. Королёв
г. колл-центр
г. Альметьевск (Республика Татарстан)
г. Кировск
г. Дудинка
г. Заринск (Алтайский край)
г. Богородицк (Тульская обл.)
г. с.Актаныш
г. Костомукша
г. Старая Купавна
г. Северодвинск (Архангельская обл.)
г. нижний новгород
г. Протвино
г. Краснотурьинск
г. Приморско-Ахтарск (Краснодарский край)
г. Ачинск (Красноярский край)
г. Белово (Кемеровская обл.)
г. Красноусольский
г. Енисейск
г. Алейск
г. Серов (Свердловская обл.)
г. Зеленокумск
г. Искитим
г. Чалтырь
г. Всеволожск (Ленинградская обл.)
г. Орёл
г. Яр-Сале
г. Омский (Омская обл.)
г. великие луки
г. королёв
г. Заполярный
г. Березники
г. Переславль-Залесский
г. Ермаковское (Красноярский к

г. Гоа (Москва)
г. Хельсинки, Финляндия (Москва)
г. Ташкент (Москва)
г. Краснодар (Россия)
г. Белоусово
г. Уфа (Ульяновск)
г. Силламяэ (Москва)
г. Харьков
г. Верхнеуральск
г. Большой Машляк (Малая Елга)
г. Юрга (Кемерово)
г. Иркутск (Казань)
г. Пушкин (Санкт-Петербург и область)
г. Подольск (Москва и область)
г. Хабаровск (Москва)
г. Родино
г. Кяхта
г. Абаза
г. Донской (Новомосковск)
г. Полазна
г. Балашов (Москва)
г. г. Хабаровск (Хабаровск)
г. пгт. Нарышкино (Орловская область)
г. Нововаршавка
г. Костерево
г. Пески
г. Рязань (Нижний Новгород)
г. Поспелиха
г. Спб (Москва)
г. Караганда (Москва)
г. Чулым
г. Нижний Новгород (Нижний Новгород)
г. Москва (Дмитров)
г. Бийск (Барнаул)
г. Казань (Москва)
г. Владимир (Нижний Новгород)
г. Адлер (Санкт-Петербург)
г. Челябинская область (Ставропольский край)
г. Сосногорск
г. Восход
г. Южно-Сахалинск (Сахалинская область)
г. Нижний Новгород (Москва)
г. Александров Владимирская область (Владимир)
г. Скопин
г. Медногорск (Москва)
г. Таганрог (Ростов-н

г. Кумертау
г. Фурманов
г. омск (Новосибирск)
г. ОРЕНБУРГ (Оренбургская область)
г. С.Петербург (Москва)
г. Кубань
г. Сокол
г. Морозовск
г. Сходня
г. Камешково
г. Клетская
г. Заринск
г. Серебряные Пруды
г. Балашиха (Московская обл.) (Москва и область)
г. Ликино-Дулево
г. Губкинский
г. Тюмень (Тюменская область)
г. Майкоп (Майкоп)
г. Ликино
г. Георгиевск (Ставропольский край)
г. Удачный-Москва (Республика Саха (Якутия))
г. Энгельс (Саратовская область)
г. Старый Оскол, с. Дмитриевка (Белгородская область)
г. Можга
г. Войвож
г. Озёры (Москва и область)
г. Нижняя Тура
г. Хворостянка
г. Москва (Тверь)
г. Краснослободск
г. Апрелевка (Москва и область)
г. Лакинск
г. Апатиты, (Мурманская область)
г. г.п. Пойковский (Ханты-Мансийский автономный округ)
г. полярные зори (Мурманская область)
г. Пыталово
г. Чугуевка
г. Всеволожск (Санкт-Петербург и область)
г. Пермь (Чайковский)
г. Одинцово (Москва)
г. Зеленчукская
г. Новопавловск
г. Дубна (Москва и область)
г. город Герой - Смоленск (Смоленск)
г.

г. п. Ванино Хабаровского края
г. рыбинск
г. Саратовская обл. город Маркс
г. Одинцово, М.О.
г. Раменское, МО
г. Москва & Ставрополь
г. п. Лотошино
г. Ryazan
г. Новоплатнировская
г. станица Ессентукская
г. уссурийск
г. ирутск
г. Красноармейск Московской области
г. город Иваново
г. г.Котельники
г. Кыштым, Челябинская обл
г. Линьи
г. с. Богучаны
г. КАМЧАТКА
г. Берёзовский
г. Санкт-Петербур
г. Гаврилов-Ям
г. кострома
г. Реж
г. выборг
г. серпухов
г. город Ефремов
г. БАРНАУЛ
г. Кривой Рог
г. Волгореченск
г. Ho Chi Minh City
г. St Petersburg
г. северск томской области
г. Сакнт-Петербург
г. Погар
г. п. Карла Маркса (Камешковский район)
г. ульяновск
г. владимир
г. Видное МО
г. Санкт-Петербург, пос. Песочный
г. Адлер
г. г. Щекино
г. Артёмовский Свердл.обл.
г. Краснознаменск МО
г. Ангарск/Иркутск
г. Большая Таволожка
г. Сосновый Бор, Лен. обл.
г. ялуторовск
г. Санкт-Ппетербург
г. Ростов-на-дону
г. Москва-СПб
г. Миллерово
г. Московская область, г.Дубна
г. ....
г. няндома Архангельской область Русф

г. г. Краснодар
г. Краснобродский
г. Московская область, г. Королев
г. Егорьевск, Московская область
г. Лысково Нижегородская обл.
г. МО г. Чехов
г. Ростов, Ярославской обл.
г. Хилок
г. Брянская обл.г.Дятьково
г. республика Татарстан Лениногорск
г. Верхняя салда
г. лукоянов
г. Озёрск (Челябинская область)
г. Урюпинск, Москва
г. юрья
г. петропавловск-Камчатский
г. Клин, МО
г. Бердск, новосибирской области
г. п.Новый Удмуртская Республика Воткинский район
г. АКСАЙ
г. Солнечногорск-30
г. Mscow
г. александров
г. Волжск-Москва
г. Московская обл. г. Химки
г. Северодвинск Архангельской области
г. пос.Правдинский
г. с.Парабель
г. 140205,Pobeda street,16,39,city of Voscresensk,Moscow region
г. CАнкт=Петербург
г. Заграница
г. г. Уссурийск
г. балабаново
г. Ярославская область г.Ростов
г. дорогобуж
г. Балаково Саратовской области
г. Мга
г. саянск
г. Волгоградская обл. Калач на Дону
г. Москва, Коломна
г. СанкПетербург
г. село Курья
г. кузнецк
г. Кудымкар
г. Воронж
г. St.Petersburg
г. В. Пышма
г. Че

г. Тейково
г. Республика Башкортостан, Буздякский район, с. Старый Буздяк
г. Белгородская область город Губкин
г. г. Люберцы, МО
г. г.Буй
г. Выборг Ленинградской области
г. с. Жутово 1 - е октябрьский район волгоградская область
г. Гнезно
г. пересвет
г. Железногорск Курской
г. КАзань
г. катайск
г. Московская область, Красногорский район, пос. Нахабино
г. пгт Алексеевское
г. елабуга
г. м осква
г. Петропаловск-Камчатский
г. ст.Сергиевская
г. с. Перемышль Калужской области
г. Усть-Нера,Якутия
г. Новосибисрк
г. краснотурьинск
г. ростов
г. Хабаровск, Москва
г. с.Неверовка Таврического района Омской области
г. Павловский -Посд
г. г. Нижний Новгород
г. Дзержинск, Нижегородская обл.
г. Рыбинск, Ярославская область
г. Североморск, Лиски
г. Балаково,Саратовская область
г. Украина п.Новотроицкое
г. Кмерово
г. Ленинградская область Всеволожский район д.Хапо-ое
г. г Железногорск
г. Екатеринург
г. копейск
г. Товарково
г. Валуки
г. с. Псынабо
г. Москва Остоженка
г. Тверская область п Пено
г. РС(Я),Ме

г. Королев, МО
г. БОРИСОВКА
г. Сибай, Уфа
г. пос. Ватутинки Московской области
г. Остров Псковская область
г. г. Кашин, Тверская область
г. козловка
г. М.О. п Шишкин Лес, Подольский р-н
г. с.Бижбуляк
г. г.Улан-Удэ
г. Индия
г. Мураши
г. КРЫМСК
г. г. Троицк М.О,
г. Россия, Южный ФО, Астраханская обл., Астрахань
г. МО г. Троицк
г. Магнитогосрк
г. Онохой
г. п. Илеза
г. Шиханы
г. Чехов М.О
г. азов
г. Ростовская обл.Орловский район,Пос.Красноармейский,пер.Красноармейский д.52
г. п.Комсомольский
г. г. Саратов
г. Усолье-сибисркое
г. р/п Татищево
г. Лобня МО
г. п.Ванино
г. Москва, Серафимович и Михайловка волгоградской обл
г. МО Электросталь
г. посёлок Одоев
г. п.Лазаревское
г. Санкт-Петернбург
г. сухой лог,свердловская область
г. Покровское
г. Москва-Ростов
г. Краснодарский край,п.Мостовской
г. п.Хотилово-2
г. Венгрия, но пишу по поводу г.Воронежа
г. смоленская обл., г.десногорск
г. г. Юрга Кемеровкой обл.
г. р.п. Таловая, Воронежской области
г. г. Петрозаводск
г. Мензелинск
г. Горки-25 Дмитро

г. Мрсква
г. Северо-Запад
г. Ростовский филиал
г. повсеместно
г. москва, павелецкая
г. Королёв МО
г. г. Тольятти
г. Оренбург (Орск)
г. Ектб
г. moscow
г. Владимирская обл.
г. г Уфа
г. Москва, павелецкое отделение
г. вся РФ
г. Москва-Владимир
г. СПб.
г. Краснодарский край, ст.Павловская
time: 470 ms


ВЫВОД: .str.contains() - использовать нельзя

In [17]:
#смотрю на список банков
for bank in bank:
    print(bank)

Бинбанк
Сбербанк России
ОТП Банк
Гагаринский
Тинькофф Банк
Восточный Экспресс Банк
ВТБ 24
Альфа-Банк
БИНБАНК кредитные карты
МТС Банк
Совкомбанк
Россельхозбанк
Московский Кредитный Банк
МДМ Банк
Хоум Кредит Банк
Бенифит-Банк
Уральский Банк Реконструкции и Развития
Русский Стандарт
Лето Банк
Петрокоммерц
Русфинанс Банк
Киви Банк
Ханты-Мансийский банк Открытие
Русславбанк
Татфондбанк
МАСТ-Банк
Национальный Банк «Траст»
Военно-Промышленный Банк
АктивКапитал Банк
Связной Банк
Платежный Центр
Московский Индустриальный Банк
Финансовый Стандарт
Ренессанс Кредит
Югра
ЮниКредит Банк
Банкирский Дом
Кредит Европа Банк
Тройка-Д Банк
ДельтаКредит
Сетелем Банк
РосинтерБанк
ФК Открытие (бывш. НОМОС-Банк)
Банк Москвы
Росбанк
Авангард
Внешпромбанк
Росэнергобанк
Российский Капитал
Российский Кредит
Газпромбанк
Кредит-Москва
Промсвязьбанк
Европлан Банк
Адамон Банк
РосЕвроБанк
Ситибанк
Райффайзенбанк
Нота-Банк
АйМаниБанк
Яндекс.Деньги
Объединенный Кредитный Банк
Интеркоммерц Банк
Бинбанк Мурманск
Адмиралт

#ВЫВОД: нужно удалить всё, что в скобках и "-Банк", " Банк ", " Банк", "Банк " и " банк "

In [18]:
dataset.groupby('city').count()[['bank_name']]

,bank_name
city,
г. (Красногорск),1
г. (Москва),2
г. (Россия),1
г. (Ханты-Мансийский автономный округ),1
г. -,1
...,...
г. янаул,2
г. ярославль,12
г. ярцево,2


time: 141 ms


In [19]:
dataset.groupby('bank_name').count()[['city']]

,city
bank_name,
2Т Банк,655
АБ Финанс,23
АББ,2
АМБ Банк,1
АМТ Банк,142
...,...
Юнистрим,81
ЯР-Банк,12
Яндекс.Деньги,226


time: 111 ms


#### city

In [20]:
#загружаю список городов РФ из dic.academic.ru
table=pd.read_html('http://dic.academic.ru/dic.nsf/ruwiki/685243')
table=table[1]
city_list = list(table['Город'].unique())
city_list[0:10]

#привожу список городов к нижнему регистру
for i in range(len(city_list)):
    city_list[i] = city_list[i].lower()

#сохраняю список городов в файл, что бы больше не обращаться к странице
with open('D:\\Python\\banki_resp_text\\city.txt', 'w') as f:
    for c in city_list:
        f.write(str(c) +"\n")

time: 763 ms


In [21]:
#загружаю список городов city_list из локального файла
city_list = []
with open('D:\\Python\\banki_resp_text\\city.txt', 'r') as f:
    for line in f:
        city_list.append(str(line.strip()))
city_list[0:10]

['москва',
 'санкт-петербург',
 'новосибирск',
 'екатеринбург',
 'нижний новгород',
 'самара',
 'омск',
 'казань',
 'челябинск',
 'ростов-на-дону']

time: 2.48 ms


In [22]:
import re

time: 542 µs


In [23]:
#проверяю, есть ли город в city_list в dataset['city'][i] и если есть заменяю на город из списка
count = 0
list_city_l = []
for i in dataset.index:
    for city in city_list:
        count = dataset['city'][i].lower().count(city)
        if count == 1:
            dataset['city'][i] = city             

D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


time: 5min 43s


In [24]:
#обрабатываю города
#удаляю 'г.', удаляю текст в скобках и привожу к нижнему регистру
for i in tqdm(range(len(dataset))):
    dataset.city[i] = dataset.city[i].replace('г.', '')
    dataset.city[i] = re.sub(r'\([^()]*\)', '', dataset.city[i]) #убираю текст в скобках
    dataset.city[i] = dataset.city[i].lower() #привожу к нижнему регистру

  0%|                                                                                       | 0/153499 [00:00<?, ?it/s]D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

time: 7min 50s


In [25]:
dataset.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


time: 24.8 ms


#pip install cld2-cffi
#тут я пытался установить библиотеку cld2 для распознавания языка, потому, что langdetect очень странно определяет язык,
#вероятно, нетотоые названия набраны из смеси русских и английских букв или же он сам по себе кривой

#from langdetect import detect

''''for city in tqdm(list(dataset.city)):
    if detect(city) != 'ru':
        print(city)
#не корректно определяет язык :(

In [26]:
#Сохраняю индексы, где city == 'неизвестно', итерироватья буду по ним
ind = dataset[dataset['city'] == 'неизвестно'].index
len(ind)

15174

time: 30.6 ms


In [28]:
#заменяю city == 'неизвестно' на найденное в title.
city_list.remove('чита') # слово "чита" содержиться в словах "читал", "читала" и т.д. пожертвую этим городом из-за ложны обнаружений
for i in ind:
    for c in city_list:
        if dataset['title'][i].lower().find(c) >= 0:
            dataset['city'][i] = c

D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


time: 56.5 s


In [40]:
#оставляю только русские и английские буквы, знаки "-" и "."
for i in tqdm(range(len(dataset))):
    dataset['city'][i] = re.sub(r'[^А-Яа-яA-Za-z-. ]+', '', dataset['city'][i])
dataset['city'].fillna('неизвестно', inplace = True)

  0%|                                                                                       | 0/153499 [00:00<?, ?it/s]D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|█████████████████████████████████████████████████████████████████████████| 153499/153499 [03:51<00:00, 662.50it/s]

time: 3min 51s


In [32]:
len(dataset[dataset['city'] == 'неизвестно'].index)

11476

time: 18 ms


In [33]:
#заменяю city == 'неизвестно' на найденное в text.
ind = dataset[dataset['city'] == 'неизвестно'].index
for i in ind:
    for c in city_list:
        if dataset['text'][i].lower().find(c) >= 0:
            dataset['city'][i] = c

time: 46 s


In [41]:
len(dataset[dataset['city'] == 'неизвестно'].index)

11478

time: 19.2 ms


In [35]:
#постобработка: 1) удаляю пробелы в начале и конце названия названия и 2) удаляю, cодержимое после ','
for i in tqdm(dataset.index):
    dataset.city[i] = dataset.iloc[i]['city'].strip().split(",")[0]     

  0%|                                                                                       | 0/153499 [00:00<?, ?it/s]D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|█████████████████████████████████████████████████████████████████████████| 153499/153499 [04:35<00:00, 556.21it/s]

time: 4min 35s


In [36]:
dataset.city.value_counts(normalize=False, sort=True)

москва             57949
санкт-петербург    15172
неизвестно         11478
новосибирск         2526
екатеринбург        2484
                   ...  
карачев                1
кемь                   1
таврическое            1
кадниково              1
сан-франциско          1
Name: city, Length: 2662, dtype: int64

time: 25 ms


#### banki

In [37]:
dataset.bank_name.fillna('неизвестно', inplace = True)

time: 6.39 ms


In [38]:
len(dataset.bank_name.unique())

670

time: 16 ms


In [39]:
#обрабатываю банки
#удаляю '-Банк', 'Банк', удаляю текст в скобках и привожу к нижнему регистру
for i in tqdm(range(len(dataset))):
    dataset.bank_name[i] = dataset.bank_name[i].replace('-Банк', '')
    dataset.bank_name[i] = dataset.bank_name[i].replace('Банк', '')
    dataset.bank_name[i] = re.sub(r'\([^()]*\)', '', dataset.bank_name[i]) #убираю текст в скобках
    dataset.bank_name[i] = dataset.iloc[i]['bank_name'].strip().split(",")[0]  
    dataset.bank_name[i] = re.sub(r'[^А-Яа-яёA-Za-z-. ]+', '', dataset.bank_name[i]) #оставляю только русские и английские буквы, знаки "-" и "."
    dataset.bank_name[i] = dataset.bank_name[i].lower() #привожу к нижнему регистру

  0%|                                                                                       | 0/153499 [00:00<?, ?it/s]D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

time: 13min 41s


In [42]:
#оставляю только русские и английские буквы, знаки "-" и "."
for i in tqdm(range(len(dataset))):
    dataset.bank_name[i] = re.sub(r'[^А-Яа-яA-Za-z-. ]+', '', dataset.bank_name[i])

  0%|                                                                                       | 0/153499 [00:00<?, ?it/s]D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|█████████████████████████████████████████████████████████████████████████| 153499/153499 [04:35<00:00, 557.44it/s]

time: 4min 35s


In [43]:
len(dataset.bank_name.unique())

666

time: 40 ms


In [44]:
dataset.bank_name.value_counts(normalize=False, sort=True)

сбербанк россии        26327
альфа                  10224
втб                     8185
русский стандарт        7943
хоум кредит             7549
                       ...  
курган                     1
камабанк                   1
единый строительный        1
град                       1
алмазэргиэнбанк            1
Name: bank_name, Length: 666, dtype: int64

time: 39.9 ms


В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру  𝑡𝑓−𝑖𝑑𝑓 , для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)

In [73]:
regex = re.compile('[А-Яа-яё]+')
def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

time: 997 µs


In [47]:
#Убираем стоп-слова:
from nltk.corpus import stopwords
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']

print(mystopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [48]:
def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

time: 2 ms


In [49]:
#Лемматизирую
from pymystem3 import Mystem
m = Mystem()

def lemmatize(text, mystem = m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

time: 65.6 ms


In [50]:
mystoplemmas = ['который', 'прошлый', 'сей', 'свой', 'наш', 'мочь', 'год']
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

dataset.title = dataset.title.apply(remove_stoplemmas)

time: 283 ms


In [95]:
def get_topic_to_tokens(df):
    tokens_by_topic = {}
    for topic in set(df['bank_name']):
        
        # берём только относительно большие темы
        if df['bank_name'].value_counts()[topic] > 100:
            
            # берём по сто случайных текстов из каждой темы
            sample = df[df['bank_name']==topic].sample(n=100)
            
            # предобрабатываем
            sample.text = sample.text.str.lower()
            sample.text = sample.text.apply(words_only)
            sample.text = sample.text.apply(remove_stopwords) 
            sample.text = sample.text.apply(lemmatize)
            sample.text = sample.text.apply(remove_stoplemmas) 

            tokens_by_topic[topic] = [tok for text in sample.text for tok in text.split()]
            
    return tokens_by_topic

time: 998 µs


In [52]:
rg1 = dataset[dataset['rating_grade'] == 1]
rg5 = dataset[dataset['rating_grade'] == 5]

time: 33.3 ms


In [70]:
rg1.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
19,москва,False,Некомпетентность сотрудников банка,2,лицензия № 1481,agelun,сбербанк россии,2015-06-05 15:00:54,открыт вклад и счет в usd. плюс к этому есть з...,1.0
33,ростов-на-дону,False,Навязывание страховки,1,лицензия № 1481,Joker888,сбербанк россии,2015-06-05 21:35:40,доброго времени! вчера мне поступило смс-уведо...,1.0
76,санкт-петербург,False,Нарушение договора банковского вклада,0,лицензия № 3006,qwarta45,активкапитал,2015-06-06 23:42:39,"05.06.2015г. около 15 часов, пришел в указанны...",1.0
85,москва,False,"Невозможно оплатить отчисления на капремонт ""Т...",4,лицензия № 1481,721,сбербанк россии,2015-06-04 13:27:04,для оплаты коммунальных платежей пользуюсь пла...,1.0
94,чапаевск,False,Некомпетентность сотрудника Сбербанка при пров...,16,лицензия № 1481,usmanovaket,сбербанк россии,2015-06-03 16:00:12,в апреле этого года пришла в отделение сбербан...,1.0


time: 10.9 ms


Оставляем только слова:

In [74]:
rg1.text = rg1.text.str.lower()
rg5.text = rg5.text.str.lower()
rg1.text = rg1.text.apply(words_only)
rg5.text = rg5.text.apply(words_only)

time: 5.44 s


In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

time: 996 µs


In [96]:
tokens_by_topic = get_topic_to_tokens(rg1)

time: 1h 37min 29s


In [97]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0)
tfidf_matrix =  tfidf.fit_transform([' '.join(tokens) for topic, tokens in tokens_by_topic.items()])
feature_names = tfidf.get_feature_names() 
dense = tfidf_matrix.todense()

time: 9.96 s


In [99]:
topic_id = 5

time: 997 µs


In [100]:
text = dense[topic_id].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(text)), text) if pair[1] > 0]
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)

time: 227 ms


In [101]:
tfidf_ranking = []
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:40]:
    tfidf_ranking.append(phrase)

time: 15.6 ms


### Постройте гистограмы длин слов в символах и в словах

In [ ]:
#создаю один большой текст из отзывов
testfull = list(dataset.text)

##### Предобработка

In [ ]:
import re
regex = re.compile("[А-Яа-я]+")

In [ ]:
#оставляю только текст
testfull = str(testfull).lower()
testfull = words_only(testfull)
testfull[0:1000]

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

sampel = testfull[0:10000]

In [ ]:
lemmata = [lemma for text in sampel for lemma in sampel.split()]
cnt = Counter(lemmata)
for i in cnt.most_common(10):
    print(i)

In [ ]:
sampellemm = mystem_analyzer.lemmatize(sampel)
cntlemm = Counter(sampellemm)
cntlemm

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
sampelsw = list()
sampelsw = [word for word in sampellemm if word not in stopwords.words('russian')]
Counter(sampelsw)

In [ ]:
sampelsw[0]

##### Закон Ципфа

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

freqs = list(cnt.values())
freqs = sorted(freqs, reverse = True)

fig, ax = plt.subplots()
ax.plot(freqs[:300], range(300))
plt.show()

#### Закон Хипса

In [ ]:
from nltk.tokenize import word_tokenize
from numpy  import array

In [ ]:
datasempel = dataset.iloc[0:1000]
datasempel.head()

In [ ]:
cnt1 = Counter()
n_words = []
n_tokens = []
tokens = []
for index, row in tqdm(datasempel.iterrows(), total = len(datasempel)):
    tokens = word_tokenize(row['text'])
    cnt.update([token for token in tokens])
    n_words.append(len(cnt1))
    n_tokens.append(sum(cnt1.values()))

In [ ]:
n_tokens = list(n_tokens[0])
fig, ax = plt.subplots()
ax.plot(n_tokens, n_words)
plt.show()

## Часть 2. Тематическое моделирование [20/100]

1. Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
2. Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?

Эта часть задания может быть сделана с использованием gensim. 

In [ ]:
dataset['title'].value_counts()

In [ ]:
for topic in set(dataset['title'].value_counts()):
    print(topic)

In [ ]:
tokens_by_topic = get_topic_to_tokens(dataset)
tokens_by_topic.keys()

In [ ]:
%%time 
dataset['title'] = dataset['title'].apply(lemmatize)

In [ ]:
dataset['title'].head()

Извлекаем биграммы по разным мерам связности:

In [ ]:
from nltk.collocations import *

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures() # класс для мер ассоциации биграм
finder = BigramCollocationFinder.from_words(dataset['text']) # класс для хранения и извлечения биграм

In [ ]:
N_best = 100 # число извлекаемых биграм

In [ ]:
%%time 
finder.apply_freq_filter(3) # избавимся от биграм, которые встречаются реже трех раз

# выбираем топ-100 биграм по каждой мере
raw_freq_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.raw_freq, N_best)]
tscore_ranking = [' '.join(i) for i in finder.nbest(bigram_measures.student_t, N_best)]
pmi_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.pmi, N_best)]
chi2_ranking =  [' '.join(i) for i in finder.nbest(bigram_measures.chi_sq, N_best)]

In [ ]:
#результаты
rankings = pd.DataFrame({
    'chi2': chi2_ranking,
    't-score' : tscore_ranking,
    'pmi': pmi_ranking,
    'raw_freq':raw_freq_ranking
})
rankings = rankings[['raw_freq', 'pmi', 't-score', 'chi2']]
rankings.head(10)

In [ ]:
from gensim.summarization import keywords

In [ ]:
dataset['bank_name'].head()

In [ ]:
tokens_by_topic['Хороший банк']

In [ ]:
topic_texts = tokens_by_topic['сбербанк россии']

In [ ]:
%%time
text = ' '.join(topic_texts)
kw = keywords(text)  

In [ ]:
%%time
text = ' '.join(topic_texts)
kw = keywords(text)

In [ ]:
rankings = pd.DataFrame({'Text Rank': kw.split('\n')})
rankings.head(10)

## Часть 3. Классификация текстов [40/100]

Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5). 

1.  Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте ```sklearn.model_selection.train_test_split``` для разделения множества отобранных документов на обучающее и тестовое. 
2. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных $n$-грам. 
3. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:
* 1-ый вариант: $tf-idf$ преобразование (```sklearn.feature_extraction.text.TfidfTransformer```) и сингулярное разложение (оно же – латентый семантический анализ) (```sklearn.decomposition.TruncatedSVD```), 
* 2-ой вариант: тематические модели LDA (```sklearn.decomposition.LatentDirichletAllocation```). 

Используйте accuracy и F-measure для оценки качества классификации. 

Ниже написан примерный Pipeline для классификации текстов. 

Эта часть задания может быть сделана с использованием sklearn. 

In [120]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# !!! На каждом этапе Pipeline нужно указать свои параметры
# 1-ый вариант: tf-idf + LSI
# 2-ой вариант: LDA

# clf = Pipeline([
#     ('vect', CountVectorizer(analyzer = 'char', ngram_range={4,6})),
#     ('clf', RandomForestClassifier()),
# ])



clf = Pipeline([ 
    ('vect', CountVectorizer(stop_words = stopwords.words('russian'), ngram_range={4,6})), 
    ('tfidf', TfidfTransformer()), 
    ('tm', TruncatedSVD()), 
    ('clf', RandomForestClassifier(n_jobs = -1))
])


time: 3.98 ms


Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение).

In [122]:
rg1 = dataset[dataset['rating_grade'] == 1]
rg5 = dataset[dataset['rating_grade'] == 5]
df = pd.concat([rg1, rg5], ignore_index=True)
x = df['text']
y = df['rating_grade']

time: 248 ms


In [124]:
x.describe()

count                                                 62100
unique                                                62098
top       05.03.2015 года в 13:47:24 у м. Нагатинская в ...
freq                                                      2
Name: text, dtype: object

time: 1.61 s


In [125]:
y.describe()

count    62100.000000
mean         1.947697
std          1.700796
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          5.000000
Name: rating_grade, dtype: float64

time: 107 ms


In [126]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

time: 9.04 ms


In [127]:
clf.fit(x_train, y_train)

D:\Program_Files\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range={4, 6}, preprocessor=None,
                                 stop_words=['и', 'в', 'во', 'не', 'что', 'он',
                                             'на', 'я', 'с', 'со', 'как', 'а',
                                             'то', 'все', 'она', 'так', 'его',...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                             

time: 15min 17s


In [128]:
y_preds = clf.predict(x_test)

time: 3min 23s


In [131]:
from sklearn.metrics import f1_score

Error in callback <function LineWatcher.stop at 0x0000015612F0EDC8> (for post_run_cell):


AssertionError: 

In [134]:
mean_f1 = f1_score(y_test, y_preds, average='binary')

time: 93.1 ms


In [135]:
mean_f1

0.8560143183361106

time: 2 ms
